# Проект e-commerce: вариант 1

In [56]:
import pandas as pd

In [57]:
customers = pd.read_csv('https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/QXrVDUoQvFhIcg')
orders = pd.read_csv('https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/0zWmh1bYP6REbw', parse_dates=[3,4,5,6,7])
order_items = pd.read_csv('https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/xCsQ0FiC-Ue4Lg')

In [58]:
items.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


Для решения задачи проведи предварительное исследование данных и сформулируй, что должно считаться покупкой. Обосновать свой выбор ты можешь с помощью фактов оплат, статусов заказов и других имеющихся данных.

Уникальные статусы заказов:

- created — создан
- approved — подтверждён
- invoiced — выставлен счёт
- processing — в процессе сборки заказа
- shipped — отгружен со склада
- delivered — доставлен пользователю
- unavailable — недоступен
- canceled — отменён

In [59]:
items.query('order_status =="delivered"').order_status.count() - items.query('order_status =="delivered"').order_delivered_customer_date.count()

8

In [60]:
items.query('order_status =="delivered"').order_status.count() - items.query('order_status =="delivered"').order_approved_at.count()

14

In [61]:
items.query('order_status =="delivered"').order_status.count() - items.query('order_status =="delivered"').order_delivered_carrier_date.count()

2

In [62]:
items.query('order_status =="unavailable"').order_approved_at.count()

609

Выше провела предварительное исследование данных.

Обнаружено:

- 8 заказов со статусом "доставлено", но отсутствующей датой доставки (техническая ошибка или, возможно, потому что курьер не зафиксировал дату доставки);
- 14 заказов со статусом "доставлено", но отсутствующей датой и временем подтверждения оплаты заказа (техническая ошибка - не проставлялась дата оплаты, или, возможно, это товары в подарок, по акции, промокоду, за баллы);
- 2 заказа со статусом "доставлено", но отсутствующей датой и временем передачи заказа в логистическую службу (возможно, самовывоз);
- 609 заказов со статусом "недоступен", но с уже подтвержденной оплатой.

Заказы, которые считаем покупками: те, что со статусом "подтверждён", "в процессе сборки заказа", "отгружен со склада", "доставлен пользователю". Остальные статусы не означают совершенную покупку.

In [63]:
#Заказы, которые считаем покупками
purchase = items[(items.order_status == 'approved') | (items.order_status == 'processing')| (items.order_status == 'shipped') | (items.order_status == 'delivered')]

# Задание 1

Сколько у нас пользователей, которые совершили покупку только один раз? 

In [64]:
df_task1 = purchase.merge(customers, on='customer_id')

In [65]:
df_task1.groupby('customer_unique_id')['order_id'].count()

customer_unique_id
0000366f3b9a7992bf8c76cfdf3221e2    1
0000b849f77a49e4a4ce2b2a4ca5be3f    1
0000f46a3911fa3c0805444483337064    1
0000f6ccb0745a6a4b88665a16c9f078    1
0004aac84e0df4da2b147fca70cf8255    1
                                   ..
fffcf5a5ff07b0908bd4e2dbc735a684    1
fffea47cd6d3cc0a88bd621562a9d061    1
ffff371b4d645b6ecea244b27531430a    1
ffff5962728ec6157033ef9805bacc48    1
ffffd2657e2aad2907e67c3e9daecbeb    1
Name: order_id, Length: 94688, dtype: int64

In [66]:
(df_task1.groupby('customer_unique_id')['order_id'].count() == 1).sum()

91816

Ответ: 91816 пользователей совершили покупку только один раз.

# Задание 2

Сколько заказов в месяц в среднем не доставляется по разным причинам (вывести детализацию по причинам)?

In [67]:
#Пропущенные значения
purchase.isna().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                  14
order_delivered_carrier_date      305
order_delivered_customer_date    1418
order_estimated_delivery_date       0
dtype: int64

In [68]:
#Изменили тип данных
purchase['order_purchase_timestamp'] = pd.to_datetime(purchase['order_purchase_timestamp'])

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [69]:
#считаем сколько всего месяцев в целом
(purchase.order_purchase_timestamp.max().to_period('M') - purchase.order_purchase_timestamp.min().to_period('M'))

<24 * MonthEnds>

In [70]:
#делим на количество месяцев
purchase[purchase.order_status!='delivered']['order_status'].value_counts()/24

shipped       46.125000
processing    12.541667
approved       0.083333
Name: order_status, dtype: float64

Ответ.
 В среднем не доставляется по разным причинам в месяц:
- 46.125 заказов отгружается со склада (останавливается на этом этапе, далее не доставлено)
- 12.541667 заказов в процессе сборки (останавливается на этом этапе, далее не отгружено) 
- 0.083333 заказов подтверждено (останавливается на этом этапе, далее не отправлено на сборку)

# Задание 3

По каждому товару определить, в какой день недели товар чаще всего покупается. 

In [71]:
df_task3 = purchase.merge(orders, on = 'order_id')[['order_approved_at', 'product_id']]
df_task3

,order_approved_at,product_id
0,2017-10-02 11:07:15,87285b34884572647811a353c7ac498a
1,2018-07-26 03:24:27,595fac2a385ac33a80bd5114aec74eb8
2,2018-08-08 08:55:23,aa4383b373c6aca5d8797843e5594415
3,2017-11-18 19:45:59,d0b61bfb1de832b15ba9d266ca96e5b0
4,2018-02-13 22:20:29,65266b2da20d04dbe00c5c2d3bb7859e
...,...,...
111737,2018-02-06 13:10:37,f1d4ce8c6dd66c47bbaa8c6781c2a923
111738,2017-08-27 15:04:16,b80910977a37536adeddd63663f916ad
111739,2018-01-08 21:36:21,d1c427060a0f73f6b889a5c7c61f2ac4
111740,2018-01-08 21:36:21,d1c427060a0f73f6b889a5c7c61f2ac4


In [72]:
#создаем колонку с днем недели, в который была совершена покупка
df_task3['weekday'] = pd.to_datetime(df_task3['order_approved_at']).dt.day_name()
df_task3

,order_approved_at,product_id,weekday
0,2017-10-02 11:07:15,87285b34884572647811a353c7ac498a,Monday
1,2018-07-26 03:24:27,595fac2a385ac33a80bd5114aec74eb8,Thursday
2,2018-08-08 08:55:23,aa4383b373c6aca5d8797843e5594415,Wednesday
3,2017-11-18 19:45:59,d0b61bfb1de832b15ba9d266ca96e5b0,Saturday
4,2018-02-13 22:20:29,65266b2da20d04dbe00c5c2d3bb7859e,Tuesday
...,...,...,...
111737,2018-02-06 13:10:37,f1d4ce8c6dd66c47bbaa8c6781c2a923,Tuesday
111738,2017-08-27 15:04:16,b80910977a37536adeddd63663f916ad,Sunday
111739,2018-01-08 21:36:21,d1c427060a0f73f6b889a5c7c61f2ac4,Monday
111740,2018-01-08 21:36:21,d1c427060a0f73f6b889a5c7c61f2ac4,Monday


In [73]:
df_task3.groupby(['product_id', 'weekday'])['weekday'].count().sort_values().groupby(level=0).tail(1)

product_id                        weekday  
00066f42aeeb9f3007548bb9d3f33c38  Sunday         1
9b8fae0d687fadb80a75aef40ea60fc2  Tuesday        1
9b9005cfa3747d9fa0d544fc9f994c14  Wednesday      1
9b90cdb2d9223fc2e400fcdda43bdd48  Sunday         1
9ba7a2e6df6dbaa83fe78f9b4ea0b3fa  Friday         1
                                              ... 
368c6c730842d78016ad823897a372db  Thursday      80
99a4788cb24856965c36a24e339b6058  Tuesday       82
422879e10f46682990de24d770e7f83d  Tuesday       89
53b36df67ebb7c41585e8d54d6772e08  Tuesday      105
aca2eb7d00ea1a7b8ebd4e68314663af  Tuesday      119
Name: weekday, Length: 32613, dtype: int64

In [74]:
def weekmax(x):
    return x.value_counts().index[0]

In [75]:
result_3 = pd.DataFrame(df_task3.dropna().groupby('product_id')['weekday'].apply(weekmax))
#ответ
result_3

,weekday
product_id,
00066f42aeeb9f3007548bb9d3f33c38,Sunday
00088930e925c41fd95ebfe695fd2655,Tuesday
0009406fd7479715e4bef61dd91f2462,Friday
000b8f95fcb9e0096488278317764d19,Friday
000d9be29b5207b54e86aa1b1ac54872,Tuesday
...,...
fff6177642830a9a94a0f2cba5e476d1,Monday
fff81cc3158d2725c0655ab9ba0f712c,Monday
fff9553ac224cec9d15d49f5a263411f,Saturday


#  Задание 4

Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)?

In [76]:
df_task4 = purchase.merge(customers, on = 'customer_id')
df_task4['year_month'] = pd.to_datetime(df_task4['order_approved_at']).dt.strftime('%Y-%m')

In [77]:
df_task4 = df_task4.groupby(['customer_unique_id', 'year_month'], as_index=False)['order_approved_at'].count().sort_values(['customer_unique_id', 'year_month'])
df_task4.head()

,customer_unique_id,year_month,order_approved_at
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05,1
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05,1
2,0000f46a3911fa3c0805444483337064,2017-03,1
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10,1
4,0004aac84e0df4da2b147fca70cf8255,2017-11,1


In [78]:
def weeks_in_month(row):
    import calendar
    year = int(row.split('-')[0])
    month = int(row.split('-')[1])
    return calendar.monthrange(year, month)[1]/7

In [79]:
#создаем колонку с количеством недель в месяце
df_task4['weeks_in_month'] = df_task4['year_month'].apply(weeks_in_month)

In [80]:
df_task4['average_purchase'] = df_task4['order_approved_at']/df_task4['weeks_in_month']
#ответ
df_task4

,customer_unique_id,year_month,order_approved_at,weeks_in_month,average_purchase
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05,1,4.428571,0.225806
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05,1,4.428571,0.225806
2,0000f46a3911fa3c0805444483337064,2017-03,1,4.428571,0.225806
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10,1,4.428571,0.225806
4,0004aac84e0df4da2b147fca70cf8255,2017-11,1,4.285714,0.233333
...,...,...,...,...,...
96538,fffcf5a5ff07b0908bd4e2dbc735a684,2017-06,1,4.285714,0.233333
96539,fffea47cd6d3cc0a88bd621562a9d061,2017-12,1,4.428571,0.225806
96540,ffff371b4d645b6ecea244b27531430a,2017-02,1,4.000000,0.250000
96541,ffff5962728ec6157033ef9805bacc48,2018-05,1,4.428571,0.225806


# Задание 5

Используя pandas, проведи когортный анализ пользователей. В период с января по декабрь выяви когорту с самым высоким retention на 3й месяц.

In [81]:
df_task5 = purchase.merge(customers, on = 'customer_id')
df_task5['year_month'] = pd.to_datetime(df_task5['order_approved_at'])

In [82]:
df_task5 = df_task5[['customer_unique_id', 'year_month']].dropna()
df_task5

,customer_unique_id,year_month
0,7c396fd4830fd04220f754e42b4e5bff,2017-10-02 11:07:15
1,af07308b275d755c9edb36a90c618231,2018-07-26 03:24:27
2,3a653a41f6f9fc3d2a113cf8398680e8,2018-08-08 08:55:23
3,7c142cf63193a1473d2e66489a9ae977,2017-11-18 19:45:59
4,72632f0f9dd73dfee390c9b22eb56dd6,2018-02-13 22:20:29
...,...,...
97883,6359f309b166b0196dbf7ad2ac62bb5a,2017-03-09 09:54:05
97884,da62f9e57a76d978d02ab5362c509660,2018-02-06 13:10:37
97885,737520a9aad80b3fbbdad19b66b37b30,2017-08-27 15:04:16
97886,5097a5312c8b157bb7be58ae360ef43c,2018-01-08 21:36:21


In [83]:
df_task5['cohort'] = df_task5.groupby('customer_unique_id')['year_month'].transform('min')
#Создадим колонку с датой первой покупки

In [84]:
df_task5.year_month = df_task5.year_month.dt.strftime('%Y-%m')
df_task5.cohort = df_task5['cohort'].dt.strftime('%Y-%m')

In [85]:
df_task5

,customer_unique_id,year_month,cohort
0,7c396fd4830fd04220f754e42b4e5bff,2017-10,2017-09
1,af07308b275d755c9edb36a90c618231,2018-07,2018-07
2,3a653a41f6f9fc3d2a113cf8398680e8,2018-08,2018-08
3,7c142cf63193a1473d2e66489a9ae977,2017-11,2017-11
4,72632f0f9dd73dfee390c9b22eb56dd6,2018-02,2018-02
...,...,...,...
97883,6359f309b166b0196dbf7ad2ac62bb5a,2017-03,2017-03
97884,da62f9e57a76d978d02ab5362c509660,2018-02,2018-02
97885,737520a9aad80b3fbbdad19b66b37b30,2017-08,2017-08
97886,5097a5312c8b157bb7be58ae360ef43c,2018-01,2018-01


In [86]:
df_task5 = df_task5.query("'2017-01' <= cohort <= '2017-12'") \
    .groupby(["cohort", "year_month"], as_index=False) \
    .agg({"customer_unique_id": "nunique"})
#фильтруем по условиям, группируем даты и считаем кол-во уникальных айди

In [87]:
df_task5["cohort"] = pd.to_datetime(df_task5["cohort"]).dt.to_period("m")
df_task5["year_month"] = pd.to_datetime(df_task5["year_month"]).dt.to_period("m")
#превращаем в номер месяца

In [88]:
df_task5

,cohort,year_month,customer_unique_id
0,2017-01,2017-01,705
1,2017-01,2017-02,4
2,2017-01,2017-03,2
3,2017-01,2017-04,1
4,2017-01,2017-05,3
...,...,...,...
166,2017-12,2018-04,13
167,2017-12,2018-05,11
168,2017-12,2018-06,9
169,2017-12,2018-07,1


In [89]:
from operator import attrgetter
df_task5['period'] = (df_task5.year_month - df_task5.cohort).apply(attrgetter("n"))
#считаем месяц покупки, начинаем с нулевого - месяц первой покупки

In [90]:
df_task5

,cohort,year_month,customer_unique_id,period
0,2017-01,2017-01,705,0
1,2017-01,2017-02,4,1
2,2017-01,2017-03,2,2
3,2017-01,2017-04,1,3
4,2017-01,2017-05,3,4
...,...,...,...,...
166,2017-12,2018-04,13,4
167,2017-12,2018-05,11,5
168,2017-12,2018-06,9,6
169,2017-12,2018-07,1,7


In [91]:
df_task5 = df_task5.pivot(index='cohort', columns = 'period', values = 'customer_unique_id').fillna(0)
#считаем

In [92]:
df_task5

period,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19
cohort,,,,,,,,,,,,,,,,,,,
2017-01,705.0,4.0,2.0,1.0,3.0,1.0,3.0,1.0,1.0,0.0,2.0,2.0,5.0,3.0,1.0,1.0,2.0,3.0,1.0
2017-02,1662.0,4.0,4.0,3.0,7.0,2.0,4.0,3.0,1.0,3.0,2.0,5.0,2.0,3.0,2.0,1.0,1.0,4.0,0.0
2017-03,2577.0,13.0,9.0,9.0,9.0,4.0,4.0,8.0,8.0,3.0,9.0,3.0,6.0,3.0,4.0,6.0,2.0,4.0,0.0
2017-04,2287.0,14.0,5.0,4.0,6.0,6.0,8.0,6.0,8.0,4.0,5.0,2.0,1.0,1.0,2.0,2.0,3.0,0.0,0.0
2017-05,3527.0,19.0,17.0,14.0,11.0,12.0,15.0,5.0,9.0,9.0,10.0,12.0,7.0,2.0,7.0,8.0,0.0,0.0,0.0
2017-06,3102.0,14.0,11.0,12.0,8.0,10.0,11.0,7.0,4.0,7.0,10.0,11.0,5.0,4.0,6.0,0.0,0.0,0.0,0.0
2017-07,3770.0,20.0,12.0,7.0,13.0,7.0,12.0,4.0,7.0,9.0,8.0,11.0,5.0,10.0,0.0,0.0,0.0,0.0,0.0
2017-08,4136.0,28.0,15.0,11.0,16.0,22.0,11.0,11.0,5.0,7.0,10.0,7.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-09,4082.0,27.0,20.0,14.0,17.0,11.0,9.0,9.0,13.0,7.0,11.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [93]:
for row in df_task5.iterrows():
    index = row[0]
    df_task5.loc[index] = row[1]/row[1][0]

In [94]:
df_task5

period,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19
cohort,,,,,,,,,,,,,,,,,,,
2017-01,1.0,0.005674,0.002837,0.001418,0.004255,0.001418,0.004255,0.001418,0.001418,0.000000,0.002837,0.002837,0.007092,0.004255,0.001418,0.001418,0.002837,0.004255,0.001418
2017-02,1.0,0.002407,0.002407,0.001805,0.004212,0.001203,0.002407,0.001805,0.000602,0.001805,0.001203,0.003008,0.001203,0.001805,0.001203,0.000602,0.000602,0.002407,0.000000
2017-03,1.0,0.005045,0.003492,0.003492,0.003492,0.001552,0.001552,0.003104,0.003104,0.001164,0.003492,0.001164,0.002328,0.001164,0.001552,0.002328,0.000776,0.001552,0.000000
2017-04,1.0,0.006122,0.002186,0.001749,0.002624,0.002624,0.003498,0.002624,0.003498,0.001749,0.002186,0.000875,0.000437,0.000437,0.000875,0.000875,0.001312,0.000000,0.000000
2017-05,1.0,0.005387,0.004820,0.003969,0.003119,0.003402,0.004253,0.001418,0.002552,0.002552,0.002835,0.003402,0.001985,0.000567,0.001985,0.002268,0.000000,0.000000,0.000000
2017-06,1.0,0.004513,0.003546,0.003868,0.002579,0.003224,0.003546,0.002257,0.001289,0.002257,0.003224,0.003546,0.001612,0.001289,0.001934,0.000000,0.000000,0.000000,0.000000
2017-07,1.0,0.005305,0.003183,0.001857,0.003448,0.001857,0.003183,0.001061,0.001857,0.002387,0.002122,0.002918,0.001326,0.002653,0.000000,0.000000,0.000000,0.000000,0.000000
2017-08,1.0,0.006770,0.003627,0.002660,0.003868,0.005319,0.002660,0.002660,0.001209,0.001692,0.002418,0.001692,0.001451,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2017-09,1.0,0.006614,0.004900,0.003430,0.004165,0.002695,0.002205,0.002205,0.003185,0.001715,0.002695,0.000735,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [95]:
df_task5[3].idxmax()

Period('2017-05', 'M')

Ответ. Когорта с самым высоким retention относится маю 2017г.

# Задание 6

Построй RFM-сегментацию пользователей, чтобы качественно оценить свою аудиторию.
Подробно опиши, как ты создавал кластеры. Для каждого RFM-сегмента построй границы метрик recency, frequency и monetary для интерпретации этих кластеров

In [96]:
df_task6 = (purchase.merge(customers, on = 'customer_id')).merge(orders, on = 'order_id')
df_task6['order_approved_at'] = pd.to_datetime(df_task6['order_approved_at'])

In [97]:
from datetime import timedelta


last_day = df_task6.order_approved_at.max()

today = last_day + timedelta(days=1)

Считаем для каждого пользователя R - время от последней покупки пользователя до текущей даты (в днях), F - суммарное количество покупок у пользователя за всё время, M - сумма покупок за всё время.


In [98]:
RFM = df_task6 \
    .groupby('customer_unique_id') \
    .agg({'order_approved_at': lambda x: (today - x.max()).days, 'product_id': lambda x: len(x), 'price': lambda x: x.sum()}) \
    .rename(columns={'order_approved_at': 'R', 'product_id': 'F', 'price': 'M'})

In [99]:
RFM

,R,F,M
customer_unique_id,,,
0000366f3b9a7992bf8c76cfdf3221e2,117.0,1,129.90
0000b849f77a49e4a4ce2b2a4ca5be3f,119.0,1,18.90
0000f46a3911fa3c0805444483337064,542.0,1,69.00
0000f6ccb0745a6a4b88665a16c9f078,326.0,1,25.99
0004aac84e0df4da2b147fca70cf8255,293.0,1,180.00
...,...,...,...
fffcf5a5ff07b0908bd4e2dbc735a684,452.0,2,1570.00
fffea47cd6d3cc0a88bd621562a9d061,267.0,1,64.89
ffff371b4d645b6ecea244b27531430a,574.0,1,89.90


Создаю ниже 4 кластера по квантилям. Каждый квантиль содержит 20% пользователей по каждому параметру RFM

In [100]:
RFM_seg = RFM.quantile(q=[0.20, 0.40, 0.60, 0.80]).to_dict()

In [101]:
RFM_seg
#Сначала я получаю квантиль для каждого параметра

{'R': {0.2: 98.0, 0.4: 182.0, 0.6: 273.0, 0.8: 388.0},
 'F': {0.2: 1.0, 0.4: 1.0, 0.6: 1.0, 0.8: 1.0},
 'M': {0.2: 39.9, 0.4: 69.9, 0.6: 109.9, 0.8: 179.9}}

In [107]:
#здесь каждому пользователю я присваиваю номер, который показывает к какому сегменту относится пользователь: от 1 до 5.
def to_seg (a,b,c):
    if a <= c[b][0.20]:
        return 1
    elif a <= c[b][0.40]:
        return 2
    elif a <= c[b][0.60]:
        return 3
    elif a <= c[b][0.80]:
        return 4
    else:
        return 5

In [108]:
#теперь получаются RFM-показатели для каждого пользователя
RFM['R_seg'] = RFM['R'].apply(to_seg, args=('R', RFM_seg, ))
RFM['F_seg'] = RFM['F'].apply(to_seg, args=('F', RFM_seg, ))
RFM['M_seg'] = RFM['M'].apply(to_seg, args=('M', RFM_seg, ))
RFM['R_seg'] = 6 - RFM['R_seg']

In [109]:
#здесь объединяются оценки R, F и M в сегменты RFM.
RFM['RFMseg'] = RFM.R_seg.map(str) + RFM.F_seg.map(str) + RFM.M_seg.map(str)

In [105]:
RFM

,R,F,M,R_seg,F_seg,M_seg,RFMseg
customer_unique_id,,,,,,,
0000366f3b9a7992bf8c76cfdf3221e2,117.0,1,129.90,4,1,4,414
0000b849f77a49e4a4ce2b2a4ca5be3f,119.0,1,18.90,4,1,1,411
0000f46a3911fa3c0805444483337064,542.0,1,69.00,1,1,2,112
0000f6ccb0745a6a4b88665a16c9f078,326.0,1,25.99,2,1,1,211
0004aac84e0df4da2b147fca70cf8255,293.0,1,180.00,2,1,5,215
...,...,...,...,...,...,...,...
fffcf5a5ff07b0908bd4e2dbc735a684,452.0,2,1570.00,1,5,5,155
fffea47cd6d3cc0a88bd621562a9d061,267.0,1,64.89,3,1,2,312
ffff371b4d645b6ecea244b27531430a,574.0,1,89.90,1,1,3,113


In [106]:
for segment in sorted(list(RFM.RFMseg.unique())):
    df_seg = RFM[RFM.RFMseg == segment]
    print('--------------------------------------------------------------')
    print('Сегмент:', segment)
    print(f'Численность группы:  {len(df_seg)}')
    print(f'Доля группы : {round(len(df_seg)/len(RFM)*100, 2)} %')
    print(f'Границы сегмента по давности : {df_seg.R.min()} - {df_seg.R.max()}')
    print(f'Границы сегмента по частоте покупок : {df_seg.F.min()} - {df_seg.F.max()}')
    print(f'Границы сегмента по сумме покупок : {df_seg.M.min()} - {df_seg.M.max()}')

--------------------------------------------------------------
Сегмент: 111
Численность группы:  3982
Доля группы : 4.21 %
Границы сегмента по давности : 389.0 - 700.0
Границы сегмента по частоте покупок : 1 - 1
Границы сегмента по сумме покупок : 3.9 - 39.9
--------------------------------------------------------------
Сегмент: 112
Численность группы:  3740
Доля группы : 3.95 %
Границы сегмента по давности : 389.0 - 700.0
Границы сегмента по частоте покупок : 1 - 1
Границы сегмента по сумме покупок : 39.92 - 69.9
--------------------------------------------------------------
Сегмент: 113
Численность группы:  3125
Доля группы : 3.3 %
Границы сегмента по давности : 389.0 - 700.0
Границы сегмента по частоте покупок : 1 - 1
Границы сегмента по сумме покупок : 69.98 - 109.9
--------------------------------------------------------------
Сегмент: 114
Численность группы:  2900
Доля группы : 3.06 %
Границы сегмента по давности : 389.0 - 700.0
Границы сегмента по частоте покупок : 1 - 1
Границы

--------------------------------------------------------------
Сегмент: 513
Численность группы:  3097
Доля группы : 3.27 %
Границы сегмента по давности : 6.0 - 98.0
Границы сегмента по частоте покупок : 1 - 1
Границы сегмента по сумме покупок : 69.95 - 109.9
--------------------------------------------------------------
Сегмент: 514
Численность группы:  3432
Доля группы : 3.62 %
Границы сегмента по давности : 6.0 - 98.0
Границы сегмента по частоте покупок : 1 - 1
Границы сегмента по сумме покупок : 109.99 - 179.9
--------------------------------------------------------------
Сегмент: 515
Численность группы:  2803
Доля группы : 2.96 %
Границы сегмента по давности : 6.0 - 98.0
Границы сегмента по частоте покупок : 1 - 1
Границы сегмента по сумме покупок : 179.99 - 6729.0
--------------------------------------------------------------
Сегмент: 551
Численность группы:  146
Доля группы : 0.15 %
Границы сегмента по давности : 7.0 - 98.0
Границы сегмента по частоте покупок : 2 - 6
Границы сегм